## SARS-CoV-2 gene mutation analysis
1. Get auspice/ncov.json from:
1. https://nextstrain.org/ncov
1. https://github.com/nextstrain/ncov
1. Run this script
1. Import to GraphXRNext/SARS-CoV-2
1. Extract (Location)-[:from_location]-(Sample)
1. Aggregate (Sum) to Location  
Weidong Yang  
2020-0301

In [1]:
# init code
%matplotlib inline 

import pandas as pd
import matplotlib.pyplot as plt
import math
import string

import json

# define functions
def parseTr(din):
    node = {}
    node['name'] = din['name']
    attrs = din['node_attrs']
    node['div'] = attrs['div']
    node['num_date'] = attrs['num_date']['value']
    attKeys=attrs.keys()
    if 'country' in attKeys:
        node['country'] = attrs['country']['value']
        node['lat']=country[attrs['country']['value']]['latitude']
        node['lon']=country[attrs['country']['value']]['longitude']
    if 'division' in attKeys: # division is not a good location indicator, it has confidence that we need to parse
        node['division'] = attrs['division']['value']
#         node['lat']=division[attrs['division']['value']]['latitude']
#         node['lon']=division[attrs['division']['value']]['longitude']
    if 'location' in attKeys:
        loc = attrs['location']['value']
        node['location'] = loc
        if loc in location.keys():
            node['lat']=location[attrs['location']['value']]['latitude']
            node['lon']=location[attrs['location']['value']]['longitude']
        else:
            print(loc, "not found")
        
    keys = din.keys()
    if 'children' in keys:
        node['children'] = [parseTr(child) for child in din['children']]
    return node


def setType(dict0):
    if 'children' in dict0.keys():
        children = dict0['children']
        for child in children:
            setType(child)
        samples = [child for child in children if 'lat' in child.keys()]
        branches = [child for child in children if 'lat' not in child.keys()]
        if len(samples) > 0:
            dict0['samples'] = samples
        if len(branches) > 0:
            dict0['branches'] = branches
        del(dict0['children'])


In [2]:
with open('data/ncov.json') as f:
  data = json.load(f)

geo_resolutions=data['meta']['geo_resolutions']

for res in geo_resolutions:
    if res['key'] == 'country':
        country = res['demes']
    elif res['key'] == 'location':
        location = res['demes']
    elif res['key'] == 'division':
        division = res['demes']
    else:
        print('key not found')

tr=data['tree']

dout=parseTr(tr)

In [3]:
setType(dout)

dout

with open('output/data.json', 'w') as outfile:
    json.dump({'branch': dout}, outfile)